In [1]:
# 卷积神经网络就是含卷积层的网络
# LeNet
# 卷积层块里的基本单位是卷积层后接最大池化层:卷积层用来识别图像里的空间模式,
# 例如线条和物体局部,之后的最大池化层则用来降低卷积层对位置的敏感性。
import gluonbook as gb
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn
import time

In [2]:
net = nn.Sequential()
net.add(
    nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120, activation='sigmoid'),
    nn.Dense(84, activation='sigmoid'),
    nn.Dense(10)
)

In [5]:
# (h - k + 1 + p + s) / s向下取整
X = nd.random.uniform(shape=(1, 1, 28, 28))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 6, 24, 24)
pool0 output shape:	 (1, 6, 12, 12)
conv1 output shape:	 (1, 16, 8, 8)
pool1 output shape:	 (1, 16, 4, 4)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


In [6]:
# 获取数据和训练
batch_size = 256
train_iter, test_iter = gb.load_data_fashion_mnist(batch_size=batch_size)

In [7]:
def try_gpu4():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

In [9]:
ctx = try_gpu4()
ctx

gpu(0)

In [10]:
def evaluate_accuracy(data_iter, net, ctx):
    acc = nd.array([0], ctx=ctx)
    for X, y in data_iter:
        X, y = X.as_in_context(ctx), y.as_in_context(ctx)
        acc += gb.accuracy(net(X), y)
    return acc.asscalar() / len(data_iter)

In [11]:
import time

In [13]:
def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
             num_epochs):
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, start = 0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
            l.backward()
            trainer.step(batch_size)
            train_l_sum += l.mean().asscalar()
            train_acc_sum += gb.accuracy(y_hat, y)
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print(('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
                'time %.1f sec' % (epoch + 1, train_l_sum / len(train_iter),
                train_acc_sum / len(train_iter),
                test_acc, time.time() - start)))

In [15]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.3190, train acc 0.100, test acc 0.098, time 4.9 sec
epoch 2, loss 1.9952, train acc 0.235, test acc 0.534, time 3.7 sec
epoch 3, loss 0.9875, train acc 0.609, test acc 0.700, time 3.3 sec
epoch 4, loss 0.7596, train acc 0.705, test acc 0.732, time 3.9 sec
epoch 5, loss 0.6741, train acc 0.734, test acc 0.758, time 3.3 sec
